### 그룹연산

In [6]:
import pandas as pd
df = pd.read_csv('./data/gapminder.tsv', sep='\t')
df

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623


In [8]:
avg_life_exp_by_year = df.groupby('year')['lifeExp'].mean()
avg_life_exp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

In [10]:
# groupby 메소드의 분할/반영/결합 과정 확인하기

# 분할 작업
years = df['year'].unique()
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

In [17]:
# 반영 작업
years_means = []
for y in years:
    m = df.loc[df.year == y, :].lifeExp.mean()
    years_means.append((y,m))
years_means

[(1952, 49.057619718309866),
 (1957, 51.50740112676056),
 (1962, 53.609249014084504),
 (1967, 55.678289577464795),
 (1972, 57.64738647887324),
 (1977, 59.57015746478874),
 (1982, 61.53319718309859),
 (1987, 63.21261267605633),
 (1992, 64.16033802816901),
 (1997, 65.01467605633802),
 (2002, 65.69492253521126),
 (2007, 67.00742253521126)]

In [15]:
# 결합 작업
df2 = pd.DataFrame({
    'year' : [y for y, m in years_means],
    '' : [m for y, m in years_means]
})
df2

,year,
0,1952,49.057620
1,1957,51.507401
2,1962,53.609249
3,1967,55.678290
4,1972,57.647386
5,1977,59.570157
6,1982,61.533197
7,1987,63.212613
8,1992,64.160338
9,1997,65.014676


In [18]:
# 사용자 함수 적용 agg() : apply와 유사
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    return sum / n

In [19]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


In [25]:
# 2개의 인자 사용
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

In [23]:
# 전체 평균 수명
global_mean = df.lifeExp.mean()
print(global_mean)

59.474439366197174


In [27]:
agg_mean_diff = df.groupby('year').lifeExp.agg(
    my_mean_diff, diff_value=global_mean)
agg_mean_diff

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

In [29]:
# 여러개의 집계 메소드 한번에 적용하기 - 리스트 
# np.count_nonzero / np.mean / np.std 적용

import numpy as np
gdf = df.groupby('year').lifeExp.agg(
    [np.count_nonzero, np.mean, np.std]
)
gdf

,count_nonzero,mean,std
year,,,
1952,142,49.057620,12.225956
1957,142,51.507401,12.231286
1962,142,53.609249,12.097245
1967,142,55.678290,11.718858
1972,142,57.647386,11.381953
1977,142,59.570157,11.227229
1982,142,61.533197,10.770618
1987,142,63.212613,10.556285
1992,142,64.160338,11.227380


In [34]:
# 여러개의 집계 메소드 한번에 적용하기 - 딕셔너리

gdf_dict = df.groupby('year').agg(
    {'lifeExp' : 'mean', 'pop' : 'median', 'gdpPercap' : 'median'}
)
gdf_dict

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334740
1972,57.647386,5877996.5,3339.129407
1977,59.570157,6404036.5,3798.609244
1982,61.533197,7007320.0,4216.228428
1987,63.212613,7774861.5,4280.300366
1992,64.160338,8688686.5,4386.085502


In [36]:
# 표준점수 계산(평균 - 표준편차)
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [38]:
# 각 연도별 lifeExp 열의 표준점수 계산
trans_z = df.groupby('year').lifeExp.transform(my_zscore)
trans_z.head()

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64

In [ ]:
# agg                      /  # transform
# year 묶어서 값을 반환    /  # 통계칼럼에서 각각의 요소들로 반환 

In [81]:
import seaborn as sns
import numpy as np

np.random.seed(1234)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10

,total_bill,tip,sex,smoker,day,time,size
187,30.46,2.00,Male,Yes,Sun,Dinner,5
179,34.63,3.55,Male,Yes,Sun,Dinner,2
31,18.35,2.50,Male,No,Sat,Dinner,4
52,34.81,5.20,Female,No,Sun,Dinner,4
71,17.07,3.00,Female,No,Sat,Dinner,3
6,8.77,2.00,Male,No,Sun,Dinner,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4
131,20.27,2.83,Female,No,Thur,Lunch,2
157,25.00,3.75,Female,No,Sun,Dinner,4
5,25.29,4.71,Male,No,Sun,Dinner,4


In [82]:
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
187,30.46,2.00,Male,Yes,Sun,Dinner,5
179,34.63,3.55,Male,Yes,Sun,Dinner,2
31,NaN,2.50,Male,No,Sat,Dinner,4
52,NaN,5.20,Female,No,Sun,Dinner,4
71,NaN,3.00,Female,No,Sat,Dinner,3
6,8.77,2.00,Male,No,Sun,Dinner,2
95,40.17,4.73,Male,Yes,Fri,Dinner,4
131,20.27,2.83,Female,No,Thur,Lunch,2
157,25.00,3.75,Female,No,Sun,Dinner,4
5,NaN,4.71,Male,No,Sun,Dinner,4


In [87]:
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [94]:
tb_grp_mean = tips_10.groupby('smoker').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = tb_grp_mean
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
187,30.46,2.00,Male,Yes,Sun,Dinner,5,30.460000
179,34.63,3.55,Male,Yes,Sun,Dinner,2,34.630000
31,NaN,2.50,Male,No,Sat,Dinner,4,18.013333
52,NaN,5.20,Female,No,Sun,Dinner,4,18.013333
71,NaN,3.00,Female,No,Sat,Dinner,3,18.013333
6,8.77,2.00,Male,No,Sun,Dinner,2,8.770000
95,40.17,4.73,Male,Yes,Fri,Dinner,4,40.170000
131,20.27,2.83,Female,No,Thur,Lunch,2,20.270000
157,25.00,3.75,Female,No,Sun,Dinner,4,25.000000
5,NaN,4.71,Male,No,Sun,Dinner,4,18.013333


In [95]:
# 데이터 변환 - apply / agg / transform

def plus(value):
    return value + 2

In [98]:
import pandas as pd
df = pd.DataFrame({'a': [10, 20, 30], 'b' : [20, 30, 40]})
df

,a,b
0,10,20
1,20,30
2,30,40


In [109]:
df.apply(plus), df.agg(plus), df.transform(plus)

(    a   b
 0  12  22
 1  22  32
 2  32  42,
     a   b
 0  12  22
 1  22  32
 2  32  42,
     a   b
 0  12  22
 1  22  32
 2  32  42)

In [108]:
df.a.apply(plus), df.a.agg(plus), df.a.transform(plus)

(0    12
 1    22
 2    32
 Name: a, dtype: int64,
 0    12
 1    22
 2    32
 Name: a, dtype: int64,
 0    12
 1    22
 2    32
 Name: a, dtype: int64)

In [120]:
# 집계 함수 사용 시

def mean(value):
    return value.mean()

In [121]:
# apply : 데이터프레임 가능 / agg : 데이터프레임, 시리즈 가능 / transform : 모두 불가

df.apply(mean), df.agg(mean), df.a.agg(mean)

(a    20.0
 b    30.0
 dtype: float64,
 a    20.0
 b    30.0
 dtype: float64,
 20.0)

In [119]:
df.a.apply(mean), df.transform(mean), df.a.transform(mean) # Error

AttributeError: 'int' object has no attribute 'min'

In [123]:
# 숫자 데이터 열과 문자 데이터 열이 함께 있는 데이터프레임

df2 = pd.DataFrame({
    'a' : [10, 20, 30, 40],
    'b' : [20, 30, 40, 50],
    'c' : ['서울', '대전', '부산', '제주'],
    'd' : ['Female', 'Male','Male', 'Female']
})

In [124]:
df2

,a,b,c,d
0,10,20,서울,Female
1,20,30,대전,Male
2,30,40,부산,Male
3,40,50,제주,Female


In [130]:
df2_grp = df2.groupby('c')

In [132]:
df2_grp.transform(plus), df2_grp.apply(mean), df2_grp.agg(mean)

C:\Users\admin\AppData\Local\Temp\ipykernel_10244\941691974.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  df2_grp.transform(plus), df2_grp.apply(mean), df2_grp.agg(mean)
C:\Users\admin\AppData\Local\Temp\ipykernel_10244\842584890.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return value.mean()
C:\Users\admin\AppData\Local\Temp\ipykernel_10244\941691974.py:1: FutureWarning: ['d'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df2_grp.transform(plus), df2_grp.apply(mean), df2_grp.agg(mean)


(    a   b
 0  12  22
 1  22  32
 2  32  42
 3  42  52,
        a     b
 c             
 대전  20.0  30.0
 부산  30.0  40.0
 서울  10.0  20.0
 제주  40.0  50.0,
        a     b
 c             
 대전  20.0  30.0
 부산  30.0  40.0
 서울  10.0  20.0
 제주  40.0  50.0)

In [ ]:
df2_grp.apply(plus), df2_grp.agg(plus) # Error

In [135]:
df2_grp.transform(mean)

C:\Users\admin\AppData\Local\Temp\ipykernel_10244\2179936544.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.transform is deprecated. In a future version, a TypeError will be raised. Before calling .transform, select only columns which should be valid for the function.
  df2_grp.transform(mean)


,a,b
0,10.0,20.0
1,20.0,30.0
2,30.0,40.0
3,40.0,50.0
